In [39]:
from pathlib import Path
import json

# import pandas as pd

from tensorflow import one_hot
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import image_dataset_from_directory

MODEL_PATH = Path.cwd() / "model_ckpts" if Path.cwd().name == "src" else Path.cwd() / "src" / "model_ckpts"

DATA_PATH = Path.cwd().parent / "data" if Path.cwd().name == "src" else Path.cwd() / "data"

print(f"Model path: {str(MODEL_PATH)}")
print(f"Data path: {str(DATA_PATH)}")

Model path: /workspaces/ds340-project/src/model_ckpts
Data path: /workspaces/ds340-project/data


In [40]:
image_size = (224, 224)
batch_size = 64

base_test_ds = image_dataset_from_directory(
    DATA_PATH / "base_input" / "test",
    seed=340,
    image_size=image_size,
    batch_size=batch_size,
)


nobg_test_ds = image_dataset_from_directory(
    DATA_PATH / "nobg_input" / "test",
    seed=340,
    image_size=image_size,
    batch_size=batch_size,
)

mod_test_ds = image_dataset_from_directory(
    DATA_PATH / "mod_input" / "test",
    seed=340,
    image_size=image_size,
    batch_size=batch_size,
)

Found 10100 files belonging to 101 classes.
Found 10100 files belonging to 101 classes.
Found 10100 files belonging to 101 classes.


In [41]:
input_shape = (*image_size, 3)
num_classes = 101

In [42]:
def one_hot_encode(image, label):
    label = tf.one_hot(label, depth=num_classes)
    return image, label

base_test_ds = base_test_ds.map(one_hot_encode)
nobg_test_ds = nobg_test_ds.map(one_hot_encode)
mod_test_ds = mod_test_ds.map(one_hot_encode)

In [43]:
models = [
    "base_model_fine",
    "base_model",
    "nobg_model_fine",
    "nobg_model",
    "mod_model_fine",
    "mod_model",
]

results = {model_name: {} for model_name in models}

In [44]:
for model_name in models:
    model = load_model(MODEL_PATH / f"{model_name}.keras", compile=True)
    print(f"Loaded model: {model_name}")
    # results[model_name]["base"] = model.evaluate(base_test_ds, return_dict=True)
    # results[model_name]["nobg"] = model.evaluate(nobg_test_ds, return_dict=True)
    # results[model_name]["mod"] = model.evaluate(mod_test_ds, return_dict=True)

    for key, test in [("base", base_test_ds), ("nobg", nobg_test_ds), ("mod", mod_test_ds)]:
        temp = model.evaluate(test, return_dict=True)
        results[model_name][f"{key}_accuracy"] = temp['accuracy']
        results[model_name][f"{key}_loss"] = temp['loss']

Loaded model: base_model_fine
158/158 ━━━━━━━━━━━━━━━━━━━━ 14s 59ms/step - accuracy: 0.8089 - loss: 0.7074
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 68ms/step - accuracy: 0.5934 - loss: 1.8292
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.5184 - loss: 2.4354
Loaded model: base_model
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 51ms/step - accuracy: 0.6894 - loss: 1.2022
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 57ms/step - accuracy: 0.4724 - loss: 2.3509
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.3708 - loss: 3.0896
Loaded model: nobg_model_fine
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 54ms/step - accuracy: 0.7666 - loss: 0.8813
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.7155 - loss: 1.1506
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 69ms/step - accuracy: 0.5173 - loss: 2.3791
Loaded model: nobg_model
158/158 ━━━━━━━━━━━━━━━━━━━━ 14s 54ms/step - accuracy: 0.6181 - loss: 1.4443
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.5783 - loss: 1.7561
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accu

In [45]:
with Path("results.json").open("w+") as f:
    json.dump(results, f, indent=4)

In [68]:
df = pd.read_json("results.json").T
df = df.drop(columns=[col for col in df.columns if "loss" in col])
df["average_accuracy"] = df[["base_accuracy", "nobg_accuracy", "mod_accuracy"]].mean(axis=1)

df = df.sort_values(by="average_accuracy", ascending=False).iloc[:3]

df = df.style.format({col: '{:.2%}' for col in df.columns})
df

,base_accuracy,nobg_accuracy,mod_accuracy,average_accuracy
mod_model_fine,74.20%,65.24%,67.29%,68.91%
nobg_model_fine,76.38%,70.99%,51.23%,66.20%
base_model_fine,80.82%,59.32%,51.58%,63.91%
